# Chap3 Deep Learning with Pytorch

## Tensors

### The creation of tensors

In [1]:
import torch
import numpy as np

In [2]:
a = torch.FloatTensor(3, 2)
a

tensor([[1.3563e-19, 1.3563e-19],
        [4.5071e+16, 2.4757e-12],
        [4.1638e-11, 8.1546e-33]])

In [3]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### GPU tensors

In [4]:
a = torch.FloatTensor([2, 3])
a

tensor([2., 3.])

In [5]:
ca = a.to('cuda')
ca

tensor([2., 3.], device='cuda:0')

In [6]:
a + 1

tensor([3., 4.])

In [7]:
ca + 1

tensor([3., 4.], device='cuda:0')

In [8]:
ca.device

device(type='cuda', index=0)

### Gradients

In [9]:
v1 = torch.tensor([1.0, 1.0], requires_grad = True)
v2 = torch.tensor([2.0, 2.0])

In [10]:
v_sum = v1 + v2
v_res = (v_sum * 2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

In [11]:
v1.is_leaf, v2.is_leaf

(True, True)

In [12]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [13]:
v1.requires_grad

True

In [14]:
v2.requires_grad

False

In [15]:
v_sum.requires_grad

True

In [16]:
v_res.requires_grad

True

In [17]:
v_res.backward()
v1.grad

tensor([2., 2.])

In [18]:
v2.grad

## NN building blocks

In [19]:
import torch.nn as nn

In [20]:
l = nn.Linear(2, 5)
v = torch.FloatTensor([1, 2])
l(v)

tensor([-1.0770, -0.2403, -0.5600, -0.8485,  1.9401], grad_fn=<AddBackward0>)

In [21]:
s = nn.Sequential(
    nn.Linear(2, 5),
    nn.ReLU(),
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1)
)
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Softmax(dim=1)
)

In [22]:
s(torch.FloatTensor([[1, 2]]))

tensor([[0.0920, 0.0920, 0.1372, 0.1307, 0.0988, 0.0818, 0.0920, 0.0920, 0.0920,
         0.0916]], grad_fn=<SoftmaxBackward>)

## Custom layers

In [23]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.pipe(x)

In [24]:
net = OurModule(num_inputs=2, num_classes=3)
v = torch.FloatTensor([[2, 3]])
out = net(v)
print(net)
print(out)
if torch.cuda.is_available():
    print("Data from cuda: %s" % out.to('cuda'))

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)
tensor([[0.2225, 0.3324, 0.4451]], grad_fn=<SoftmaxBackward>)
Data from cuda: tensor([[0.2225, 0.3324, 0.4451]], device='cuda:0', grad_fn=<CopyBackwards>)


## Monitoring with TensorBoard

## Plotting stuff

In [25]:
import math
from tensorboardX import SummaryWriter
writer = SummaryWriter()
funcs = {"sim" : math.sin, "cos" : math.cos, "tan" : math.tan}

In [26]:
for angle in range(-360, 360):
    angle_rad = angle * math.pi / 180
    for name, fun in funcs.items():
        val = fun(angle_rad)
        writer.add_scalar(name, val, angle)

writer.close()

## Example - GAN on Atari images

In [27]:
import random
import argparse
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter

import torchvision.utils as vutils

import gym
import gym.spaces

import numpy as np

In [28]:
class InputWrapper(gym.ObservationWrapper):
    """
    Preprocessing of input numpy array:
    1. resize image into predefined size
    2. move color channel axis to a first place
    """
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args)
        assert isinstance(self.observation_space, gym.spaces.Box)
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(
            self.observation(old_space.low),
            self.observation(old_space.high),
            dtype=np.float32)

    def observation(self, observation):
        # resize image
        new_obs = cv2.resize(
            observation, (IMAGE_SIZE, IMAGE_SIZE))
        # transform (210, 160, 3) -> (3, 210, 160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)

In [29]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # this pipe converges image into the single number
        self.conv_pipe = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 2, out_channels=DISCR_FILTERS * 4,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 4, out_channels=DISCR_FILTERS * 8,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 8, out_channels=1,
                      kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        conv_out = self.conv_pipe(x)
        return conv_out.view(-1, 1).squeeze(dim=1)

In [30]:
class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # pipe deconvolves input vector into (3, 64, 64) image
        self.pipe = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS * 8,
                               kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(GENER_FILTERS * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 8, out_channels=GENER_FILTERS * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 4, out_channels=GENER_FILTERS * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.pipe(x)

In [31]:
log = gym.logger
log.set_level(gym.logger.INFO)

LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

# dimension input image will be rescaled
IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

In [32]:
def iterate_batches(envs, batch_size=BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)

    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalising input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32) * 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
envs = [
    InputWrapper(gym.make(name))
    for name in ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')
]
input_shape = envs[0].observation_space.shape

net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

objective = nn.BCELoss()
gen_optimizer = optim.Adam(
    params=net_gener.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
dis_optimizer = optim.Adam(
    params=net_discr.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
writer = SummaryWriter()

gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

for batch_v in iterate_batches(envs):
    # fake samples, input is 4D: batch, filters, x, y
    gen_input_v = torch.FloatTensor(
        BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
    gen_input_v.normal_(0, 1)
    gen_input_v = gen_input_v.to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)

    # train discriminator
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_true_v, true_labels_v) + \
                objective(dis_output_fake_v, fake_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    dis_losses.append(dis_loss.item())

    # train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss_v = objective(dis_output_v, true_labels_v)
    gen_loss_v.backward()
    gen_optimizer.step()
    gen_losses.append(gen_loss_v.item())

    iter_no += 1
    if iter_no % REPORT_EVERY_ITER == 0:
        log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e",
                    iter_no, np.mean(gen_losses),
                    np.mean(dis_losses))
        writer.add_scalar(
            "gen_loss", np.mean(gen_losses), iter_no)
        writer.add_scalar(
            "dis_loss", np.mean(dis_losses), iter_no)
        gen_losses = []
        dis_losses = []
    if iter_no % SAVE_IMAGE_EVERY_ITER == 0:
        writer.add_image("fake", vutils.make_grid(
            gen_output_v.data[:64], normalize=True), iter_no)
        writer.add_image("real", vutils.make_grid(
            batch_v.data[:64], normalize=True), iter_no)

INFO: Making new env: Breakout-v0
INFO: Making new env: AirRaid-v0
INFO: Making new env: Pong-v0
INFO: Iter 100: gen_loss=5.325e+00, dis_loss=5.761e-02
INFO: Iter 200: gen_loss=6.939e+00, dis_loss=4.935e-03
INFO: Iter 300: gen_loss=7.389e+00, dis_loss=2.117e-03
INFO: Iter 400: gen_loss=7.608e+00, dis_loss=1.623e-03
INFO: Iter 500: gen_loss=9.229e+00, dis_loss=3.880e-02
INFO: Iter 600: gen_loss=8.203e+00, dis_loss=6.814e-02
INFO: Iter 700: gen_loss=6.819e+00, dis_loss=1.020e-02
INFO: Iter 800: gen_loss=6.276e+00, dis_loss=1.562e-01
INFO: Iter 900: gen_loss=5.884e+00, dis_loss=2.541e-01
INFO: Iter 1000: gen_loss=5.304e+00, dis_loss=2.822e-01
INFO: Iter 1100: gen_loss=5.924e+00, dis_loss=1.113e-01
INFO: Iter 1200: gen_loss=6.032e+00, dis_loss=1.956e-01
INFO: Iter 1300: gen_loss=5.662e+00, dis_loss=1.475e-01
INFO: Iter 1400: gen_loss=4.880e+00, dis_loss=1.836e-01
INFO: Iter 1500: gen_loss=5.680e+00, dis_loss=2.375e-01
INFO: Iter 1600: gen_loss=4.829e+00, dis_loss=1.826e-01
INFO: Iter 1700:

KeyboardInterrupt: 